In [1]:
import torch

In [1]:
import h5py
import numpy as np
import time
import os

# import filters
import Estkf
# ,Ensrf,Senkf


# import torch_filters
# TO DO

In [3]:
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())
# check if cuda is available
print(torch.cuda.is_available())

True
True
False


In [2]:
def stats(x):
    return x.shape, np.mean(x,axis=1), np.cov(x[:1000,:],ddof=1)

def close(state_analysis,xb):
    return np.allclose(state_analysis.mean(axis=1),xb.mean(axis=1))

def timeit(obs_covariance):
    """
    function execution time
    """
    st = time.time()
    x = obs_covariance._assimilate()
    et = time.time()

    # get the execution time
    elapsed_time = et - st
    
    return 'Execution time:', elapsed_time, 'seconds'

In [2]:
filename = '/Users/rylanspence/Desktop/Git/Adcirc/Adcirc_Plots/harvey_ensemble_files/example_sim_outputs.npy' 

with open(filename,'rb') as obs_covariance:
    zeta = np.load(obs_covariance)
    obs = np.load(obs_covariance)
    
    
print(obs_covariance"Ensemble state matrix dimensions (ne x nx): {zeta.shape}")
print(obs_covariance"Observation vector at one time step dimensions: (ny) {obs.shape}")

Ensemble state matrix dimensions (ne x nx): (10, 3352598)
Observation vector at one time step dimensions: (ny) (11,)


In [27]:
state_forecast = zeta.T[:,:5]                            # ensemble of state vectors
ne = state_forecast.shape[1]                        # number of ensemble members
observations = obs                                 # model observations (adcirc output file fort.63.nc)                  
model_observations = np.eye(obs.size)[:,:5]           # observation operator ( adcirc output file fort.61.nc)
print(model_observations.shape)

params = [state_forecast,model_observations,observations]

# filter_estkf = Estkf.estkf(*params)
# x1 = filter_estkf._assimilate()

(11, 5)


In [25]:
print(state_forecast.shape)

(3352598, 5)


In [20]:
ne

10

In [28]:
filter_estkf = Estkf.estkf(*params)
x1 = filter_estkf._assimilate()
# torch_estkf = ESTKFT(*params)

# x2 = torch_estkf._assimilate()

nx 3352598 ne 5 ny 11


TypeError: unsupported operand type(s) for /: 'int' and 'NoneType'

In [16]:
print(stats(x1),"\n")
# print(stats(x2),"\n")
# print(close(x1,x2),"\n")
# print(timeit(filter_estkf))
# print(timeit(torch_estkf))

NameError: name 'x1' is not defined

In [9]:
filter_ensrf = EnSRF(*params)
x3 = filter_ensrf._assimilate()
torch_ensrf = EnSRFT(*params)
x4 = torch_ensrf._assimilate()

In [10]:
print(stats(x3))
print(stats(x4),"\n")
print(close(x3,x4),"\n")
print(timeit(filter_ensrf))
print(timeit(torch_ensrf))

((3352598, 2), array([0.54154072+0.j, 0.54413802+0.j, 0.55966858+0.j, ...,
       0.30275894+0.j, 0.30275792+0.j, 0.30275961+0.j]), array([[ 1.62669040e-05+0.j,  1.33313593e-05+0.j,  9.91098353e-06+0.j,
        ..., -9.89622554e-06+0.j,  1.01836389e-05+0.j,
        -1.56405570e-06+0.j],
       [ 1.33313593e-05+0.j,  1.09255664e-05+0.j,  8.12243573e-06+0.j,
        ..., -8.11034098e-06+0.j,  8.34588736e-06+0.j,
        -1.28180436e-06+0.j],
       [ 9.91098353e-06+0.j,  8.12243573e-06+0.j,  6.03849354e-06+0.j,
        ..., -6.02950189e-06+0.j,  6.20461504e-06+0.j,
        -9.52936728e-07+0.j],
       ...,
       [-9.89622554e-06+0.j, -8.11034098e-06+0.j, -6.02950189e-06+0.j,
        ...,  6.02052363e-06+0.j, -6.19537603e-06+0.j,
         9.51517753e-07+0.j],
       [ 1.01836389e-05+0.j,  8.34588736e-06+0.j,  6.20461504e-06+0.j,
        ..., -6.19537603e-06+0.j,  6.37530661e-06+0.j,
        -9.79152419e-07+0.j],
       [-1.56405570e-06+0.j, -1.28180436e-06+0.j, -9.52936728e-07+0.j,
     

In [11]:
filter_senkf = SEnKF(*params)
x5 = filter_senkf._assimilate()
torch_senkf = SEnKFT(*params)
x6 = torch_senkf._assimilate()

In [12]:
print(stats(x5))
print(stats(x6),"\n")
print(close(x5,x6),"\n")
print(timeit(filter_senkf))
print(timeit(torch_senkf))

((3352598, 2), array([0.54390396, 0.54607479, 0.56110844, ..., 0.30093394, 0.30093293,
       0.30093462]), array([[ 3.04915212e-08,  2.49889852e-08,  1.85776571e-08, ...,
        -1.85499939e-08,  1.90887363e-08, -2.93174641e-09],
       [ 2.49889852e-08,  2.04794434e-08,  1.52251111e-08, ...,
        -1.52024401e-08,  1.56439604e-08, -2.40267999e-09],
       [ 1.85776571e-08,  1.52251111e-08,  1.13188627e-08, ...,
        -1.13020083e-08,  1.16302494e-08, -1.78623360e-09],
       ...,
       [-1.85499939e-08, -1.52024401e-08, -1.13020083e-08, ...,
         1.12851790e-08, -1.16129314e-08,  1.78357380e-09],
       [ 1.90887363e-08,  1.56439604e-08,  1.16302494e-08, ...,
        -1.16129314e-08,  1.19502025e-08, -1.83537364e-09],
       [-2.93174641e-09, -2.40267999e-09, -1.78623360e-09, ...,
         1.78357380e-09, -1.83537364e-09,  2.81886134e-10]]))
((3352598, 2), array([0.54103649, 0.54372479, 0.55936137, ..., 0.30314833, 0.30314731,
       0.303149  ]), array([[ 6.36153926e-06,  